### Pre-process pictures for *train*
1. compute **nmf** for all wav files and save as `npy` file
2. get **labels** as ground truth from `wav` files path and save as `npy` file
3. save paths of all `npy` file in `train.h5` and `val.h5`

In [ ]:
import os
import numpy as np
from dataset.NMF import audio_import

#indexes1 = ['accordion', 'acoustic_guitar', 'cello',  'flute',  'saxophone', 'trumpet', 'violin', 'xylophone']
#indexes2 = [[401], [402], [486], [558], [776], [513], [889], [642]]
#indexes3 = [[401], [402,546], [486], [558], [776], [513], [889], [642]]
idxs=[401,402,486,513,558,642,776,889]
names=['accordion','acoustic_guitar','cello','trumpet','flute','xylophone','saxophone','violin']

path = 'h:/Study/bpfile/dataset/audios/'
wav_files = []
bases_filepath = []
labels_filepath = []

# 遍历数据集 `path` 路径下所有的 wav 文件，记录他们的路径
# wav_files[0] = 'h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/1.wav'
for root,dirs,files in os.walk(path, topdown=True):
    for name in files:
        if '.wav' in name:
            wav_files.append(os.path.normcase(root+os.path.sep+name).replace('\\','/'))
            
# 计算 bases 和 labels
for i, fname in enumerate(wav_files):
    label = np.zeros(1000)   # 1000个乐器类别
    
    # 每个 wav 对应的 NMF 结果保存为 *.npy 文件，对应路径存入 bases
    # bases_filepath[0] = 'h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/1_bases.npy'
    # np.array(nmf_base[0]).shape = (2401, 16)
    nmf_base = audio_import.nmf([fname])
    np.save(fname[:-4]+'_bases'+'.npy', nmf_base[0])
    bases_filepath.append(fname[:-4]+'_bases'+'.npy')
    
    # 根据文件路径名确定当前音频中乐器的 label，作为 ground truth 训练
    # labels_filepath[0] = 'h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/1_labels.npy'
    for instrument, index in zip(names, idxs):
        if instrument in fname:
            label[index] = 1
    print(sum(label))
    np.save(fname[:-4]+'_labels'+'.npy', label)
    labels_filepath.append(fname[:-4]+'_labels'+'.npy')
    
    print(bases_filepath[-1])
    print(labels_filepath[-1])
    

### generate `train.h5` and `val.h5`

In [ ]:
import h5py
import random

dataset_num = len(wav_files)
train_num = int(0.7*dataset_num)

# 随机选取 70% 的数据作为验证集
train_bases = random.sample(range(dataset_num), train_num)

# 路径 utf-8 编码，否则报错
train_bases_encode = []
train_labels_encode = []
val_bases_encode = []
val_labels_encode = []
for i in range(dataset_num):
    if i in train_bases:
        train_bases_encode.append(bases_filepath[i].encode(encoding='utf-8', errors='strict'))
        train_labels_encode.append(labels_filepath[i].encode(encoding='utf-8', errors='strict'))
    else:
        val_bases_encode.append(bases_filepath[i].encode(encoding='utf-8', errors='strict'))
        val_labels_encode.append(labels_filepath[i].encode(encoding='utf-8', errors='strict'))

h5f = h5py.File('dataset/train.h5', 'w')
h5f.create_dataset('bases', data=train_bases_encode)
h5f.create_dataset('labels', data=train_labels_encode)
h5f.close()

h5f = h5py.File('dataset/val.h5', 'w')
h5f.create_dataset('bases', data=val_bases_encode)
h5f.create_dataset('labels', data=val_labels_encode)
h5f.close()

## train ...

### Pre-process pictures for *test*
1. compute **nmf** for all wav files and save as `npy` file
2. compute **labels** by feat_extrator and save as `npy` file
3. save paths of all `npy` file in `train.h5` and `val.h5`

In [2]:
import os
import random
import numpy as np
from dataset.NMF import audio_import
from util.feat_extractor import load_model, get_CAM, feat_pred

idxs=[401,402,486,513,558,642,776,889]
names=['accordion','acoustic_guitar','cello','trumpet','flute','xylophone','saxophone','violin']

test_path = 'h:/Study/bpfile/testset25/'
wav_files = []
img_files = []
bases_filepath = []
labels_filepath = []
locations = {}

# 寻找音频路径与图片路径
# wav_dir = 'h:/Study/bpfile/testset25/testaudio/'
# img_dir = 'h:/Study/bpfile/testset25/testimage/'
wav_dir = os.path.normcase(os.path.join(test_path, 'gt_audio/')).replace('\\','/')
img_dir = os.path.normcase(os.path.join(test_path, 'testimage/')).replace('\\','/')
'''
for cur_dir in os.listdir(test_path):
    if 'audio' in cur_dir:
        os.path.normcase(os.path.join(test_path, cur_dir, '/')).replace('\\','/')
    if 'image' in cur_dir:
        os.path.normcase(os.path.join(test_path, cur_dir, '/')).replace('\\','/')
'''
        
# 寻找所有 wav 的文件名、图片文件所在文件夹
# wav_files[0] = 'accordion_1_saxophone_1.wav'
# img_files[0] = 'accordion_1_saxophone_1'
for f in os.listdir(wav_dir):   # 剔除非 wav 文件
    if '_gt1.wav' not in f and '_gt2.wav' not in f and '.wav' in f:
        wav_files.append(f)
for f in os.listdir(img_dir):   # 剔除目录下所有文件，只保留文件夹
    if os.path.isdir(img_dir + f):
        img_files.append(f)
        locations[f+'.mp4'] = []
        
# 排序确保 wav 文件与图片文件夹相对应
wav_files.sort()
img_files.sort()
load_model()

# 计算 bases 和 labels
for wav_fname, img_folder in zip(wav_files, img_files):
    assert wav_fname[:-4] == img_folder    # 确保 wav 文件与图片文件夹相对应
    '''
    # 计算 NMF 并将结果保存在 npy 文件中
    # bases_filepath[0] = 'h:/Study/bpfile/testset25/testaudio/accordion_1_saxophone_1_bases.npy'
    # np.array(nmf_base[0]).shape = (2401, 16)
    nmf_base = audio_import.nmf([wav_dir+wav_fname])
    np.save(wav_dir+wav_fname[:-4]+'_bases'+'.npy', nmf_base[0])
    bases_filepath.append(wav_dir+wav_fname[:-4]+'_bases'+'.npy')
    '''
    imgs = []
    # 获得当前文件夹下所有图片（绝对）路径，并随机抽样
    # img_folder = 'accordion_1_saxophone_1'
    # imgs[0] = '000001.jpg'
    for img in os.listdir(img_dir+img_folder):
        if '.jpg' in img or '.png' in img:
            imgs.append(img)
    imgs = random.sample(imgs, 30)  # 由于图片数量过多，每个视频中只随机抽取 50 张图片进行预测
    
    probs=np.zeros([8])
    location = np.zeros([8])
    # 计算图像的 label，并获得每种乐器的定位（列号）
    for img in imgs:
        probs1 = feat_pred(img_dir+img_folder, img)
        probs = probs + np.array(probs1)
        #(locate, CAMs, heatmap) = get_CAM(img_dir+img_folder, 'results', img)  # heatmap 保存为文件
        locate = get_CAM(img_dir+img_folder, 'results', img)
        location = location + locate
        '''
        print(np.argmax(CAMs))
        print(CAMs)
        plt.figure()
        plt.imshow(0.3*CAMs)
        plt.figure()
        plt.imshow(heatmap)
        '''
        
    locations[img_folder+'.mp4'] = location
    #print(probs)
    print(location)
    softmax = np.zeros(1000)
    #for i in range(len(probs)):
    #    softmax[idxs[i]] = probs[i]
    #np.save(img_dir+img_folder+'/labels'+'.npy', softmax)
    labels_filepath.append(img_dir+img_folder+'/labels'+'.npy')
    
    #print(bases_filepath[-1])
    print(labels_filepath[-1])
    

d:\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


[41638. 28137. 23348. 26450. 21528. 29065. 38568. 18622.]
h:/study/bpfile/testset25/testimage/accordion_1_saxophone_1/labels.npy
[44184. 22514. 45354. 28776. 23004. 42053. 36036. 49654.]
h:/study/bpfile/testset25/testimage/accordion_2_acoustic_guitar_2/labels.npy
[ 7318. 10006. 16459. 11275. 19657. 10952.  7481. 12197.]
h:/study/bpfile/testset25/testimage/accordion_2_cello_3/labels.npy
[42947. 37449. 35432. 34241. 40664. 41247. 40478. 33705.]
h:/study/bpfile/testset25/testimage/accordion_3_acoustic_guitar_3/labels.npy
[16221. 12031. 14893. 13090. 11563. 16330. 13942. 12595.]
h:/study/bpfile/testset25/testimage/accordion_4_saxophone_3/labels.npy
[30113. 27039. 24987. 32208. 32384. 29942. 27369. 26171.]
h:/study/bpfile/testset25/testimage/accordion_5_cello_1/labels.npy
[11254. 16168. 14288. 23516. 14177. 22383. 22966. 27767.]
h:/study/bpfile/testset25/testimage/accordion_6_saxophone_2/labels.npy
[16209. 20243. 29032. 36223. 23102. 24569. 12941. 37013.]
h:/study/bpfile/testset25/testimage

In [8]:
import h5py

test_bases_encode = []
test_labels_encode = []

for i in range(len(bases_filepath)):
    test_bases_encode.append(bases_filepath[i].encode(encoding='utf-8', errors='strict'))
    test_labels_encode.append(labels_filepath[i].encode(encoding='utf-8', errors='strict'))

h5f = h5py.File('dataset/test.h5', 'w')
h5f.create_dataset('bases', data=test_bases_encode)
h5f.create_dataset('labels', data=test_labels_encode)
h5f.close()

In [11]:
import json

for k in locations.keys():
    locations[k] = list(locations[k])

with open('dataset/locations.json','w') as f:
    json.dump(locations, f)

In [2]:
from util import feat_extractor
from util.feat_extractor import feat_pred, feat_pred_by_seg, load_model

load_model()
img_dir = '.'
imgname = '000053.jpg'

print(feat_pred(img_dir,imgname))
left,right = feat_pred_by_seg(img_dir,imgname)
print('left ', left)
print('right ', right)

position = [0]*8
left_max = max(left)
right_max = max(right)
if left_max > right_max:
    position[left.index(left_max)] = 1      # 确定左侧乐器种类
    right[left.index(left_max)] = 0
    position[right.index(max(right))] = 2   # 确定右侧乐器种类
else:
    position[right.index(right_max)] = 2    # 确定右侧乐器种类
    left[right.index(right_max)] = 0
    position[left.index(max(left))] = 1     # 确定左侧乐器种类
print(position)

[4.8531398e-08, 4.203821e-10, 3.0119654e-07, 1.6685843e-09, 7.592569e-08, 0.58534557, 4.232982e-09, 8.99717e-09]
left  [3.3698166e-06, 1.5242497e-08, 1.61589e-05, 3.455999e-08, 1.1760211e-07, 2.40524e-06, 2.1710822e-08, 2.5602446e-06]
right  [2.5203516e-08, 9.114263e-12, 1.9799065e-09, 6.485805e-11, 5.8738996e-09, 0.11121938, 1.4527327e-10, 1.1737265e-10]
[0, 0, 1, 0, 0, 2, 0, 0]


In [3]:
from Evaluate import Evaluate
Evaluate('H:/Study/bpfile/testset25/result_json','H:/Study/bpfile/testset25/Result_sep','H:/Study/bpfile/testset25/gt_audio')

([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [11.025667273241673,
  -8.616144398241026,
  8.70033389108073,
  2.5079283361029905,
  9.652112032056358,
  -0.15310698486073632,
  2.05947453003453,
  5.1101977566897325,
  1.562369217599937,
  -1.3130846512270777,
  0.9041050961813324,
  12.655941904901958,
  -13.438770299126723,
  8.627968336798402,
  -8.309988122393,
  7.548632691892022,
  2.0753651542099916,
  3.9472065291720515,
  2.407020892892854,
  1.5340932165582575,
  -1.489769209593552,
  4.694007757740754,
  0.7759560881044568,
  3.236182103382213,
  9.875781822568525,
  -8.932016372898001,
  -7.911161286186349,
  6.71525418993304,
  3.171514461085306,
  -4.228833308769227,
  -3.532583959722861,
  2.446789310157182,
  -0.4158362013207336,
  3.6410206480297926,
  -7.14992673159284,
  10.78672873386352,
  4.3889315979614345,
  -5.781894478256094,
  6.211971539542357,
  -1.0298283188974704,
  -0.04018120965681371,
  -1.7705561093021722,
  -11.36801

In [9]:
# 2018/12/31 01:00
accur = [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0]
snr = [11.347610961650894,
  -9.632960200620044,
  8.70021952059402,
  2.5079513008718477,
  9.652218240462485,
  -0.1527975835486499,
  2.0595129473305334,
  5.1103995341998925,
  1.561645286855171,
  -1.313358260349896,
  0.9039634811239038,
  12.656031646268426,
  -13.439097840668804,
  8.627718589954496,
  -8.30973963605447,
  7.548450334721046,
  2.0754457673556868,
  3.9472773987273646,
  2.4069430738422453,
  1.534099362765119,
  -1.4902245012246584,
  4.6937194856242614,
  2.605547560421433,
  4.373700425100127,
  9.875911990888353,
  -8.932793377600113,
  -7.911671922013421,
  6.714576715646839,
  2.952031236943138,
  -4.591842359176853,
  -3.664687551573451,
  2.461675985931538,
  -0.4154217807845178,
  3.641413631044385,
  -6.275929831730655,
  10.901451073989607,
  4.391146147909199,
  -5.780470227883133,
  6.2859942439767735,
  -1.4655790234925066,
  -0.03865087571590465,
  -1.7699167771162454,
  -11.366997010864136,
  14.822491686259596,
  2.4814621671509456,
  -0.5059430078344603,
  8.13116251910653,
  -4.994748791387616,
  6.488218187777051,
  -0.03958338260151157]

print(sum(accur))
print(sum(snr)/len(snr))

12
1.5873515312450368


In [10]:
cnt = 0
for i in range(len(snr)):
    cnt = cnt + int(snr[i]>0)
print(cnt)

30


In [4]:
# 2018/12/31 11:00
snr = [11.025667273241673,
  -8.616144398241026,
  8.70033389108073,
  2.5079283361029905,
  9.652112032056358,
  -0.15310698486073632,
  2.05947453003453,
  5.1101977566897325,
  1.562369217599937,
  -1.3130846512270777,
  0.9041050961813324,
  12.655941904901958,
  -13.438770299126723,
  8.627968336798402,
  -8.309988122393,
  7.548632691892022,
  2.0753651542099916,
  3.9472065291720515,
  2.407020892892854,
  1.5340932165582575,
  -1.489769209593552,
  4.694007757740754,
  0.7759560881044568,
  3.236182103382213,
  9.875781822568525,
  -8.932016372898001,
  -7.911161286186349,
  6.71525418993304,
  3.171514461085306,
  -4.228833308769227,
  -3.532583959722861,
  2.446789310157182,
  -0.4158362013207336,
  3.6410206480297926,
  -7.14992673159284,
  10.78672873386352,
  4.3889315979614345,
  -5.781894478256094,
  6.211971539542357,
  -1.0298283188974704,
  -0.04018120965681371,
  -1.7705561093021722,
  -11.368012914148942,
  14.822047305213744,
  2.481380517704754,
  -0.5060000788044992,
  9.136656160609991,
  -3.96703800321657,
  6.488657374146306,
  -0.03982178185451063]

print(sum(snr)/len(snr))
cnt = 0
for i in range(len(snr)):
    cnt = cnt + int(snr[i]>0)
print(cnt)

1.5839348409877392
30
